In [28]:
#   Copyright sk3pta 2024

In [29]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.metrics import classification_report,confusion_matrix
#https://habr.com/ru/articles/802185/


In [30]:
df = pd.read_csv('diabetes.csv')

# By default, this function will create a grid of Axes such that each numeric variable in data will by shared across the y-axes across a single row and the x-axes across a single column. The diagonal plots are treated differently: a univariate distribution plot is drawn to show the marginal distribution of the data in each column.
  
#sns.pairplot(df, hue="Outcome")


x = df.drop('Outcome',axis = 1)
y = df['Outcome']



In [31]:

#sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None
"""
test_size можно в процентах
random_state  Whenever randomization is part of a Scikit-learn algorithm, a random_state parameter may be provided to control the random number generator used. N

"""

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30, random_state=37)

In [43]:
from sklearn.model_selection import GridSearchCV, cross_val_score


"""
В официальной документации библиотека SciKit Learn говорится, что гамма определяет насколько далеко каждый из элементов в наборе данных имеет влияние при определении “идеальной линии”.
"""
svc_model = SVC()
grid = GridSearchCV(svc_model, {'C': [1,10,100,1000],
                                'gamma' : [1, 0.1, 0.01, 0.001, 0.0001],
                                'kernel' : ['rbf'] }, cv=5, n_jobs=-1, verbose=True)
grid.fit(x_train,y_train)
print(f"Best params: {grid.best_params_}")
grid_pred = grid.predict(x_test)
print(confusion_matrix(y_test, grid_pred))
print(classification_report(y_test,grid_pred))

svc_model.fit(x_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
[[135  16]
 [ 38  42]]
              precision    recall  f1-score   support

           0       0.78      0.89      0.83       151
           1       0.72      0.53      0.61        80

    accuracy                           0.77       231
   macro avg       0.75      0.71      0.72       231
weighted avg       0.76      0.77      0.76       231



SVC()

In [42]:
# теперь проверяем
predict = svc_model.predict(x_test)

#print(y_test)
#print(y)

conf_mx = confusion_matrix(y_test, predict)
print(f"    Positive    Negative    ")
print(f"       {conf_mx[0][0]}         {conf_mx[0][1]}")
print(f"       {conf_mx[1][0]}         {conf_mx[1][1]}")
print()

# true positive = предсказанный и реальный выход совпали
# false positive = предсказанный результат позитивный, но целевой - нет
# и т. д
#   Pos Neg

#   TP  FP
#   FN  TN

# recall = tp / (tp + fn)
# precision = tp / (tp +fp)
# accur     = (tp + tn) / (tp + tn + fp + fn)
# f1        = ( 2 * prec * rec) / (prec + rec)

# macro avg Рассчитывает среднее метрических показателей для каждого класса, взвешивая их одинаково:
# weighted avg Рассчитывает среднее метрических оценок для каждого класса, взвеши­вая их пропорционально представительству в сведениях:

print(classification_report(y_test, predict))

    Positive    Negative    
       139         12
       38         42

              precision    recall  f1-score   support

           0       0.79      0.92      0.85       151
           1       0.78      0.53      0.63        80

    accuracy                           0.78       231
   macro avg       0.78      0.72      0.74       231
weighted avg       0.78      0.78      0.77       231



In [34]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz


In [35]:
# x_train x_test y_train y_test оставил 

dec_tree = DecisionTreeClassifier(max_depth=5,random_state=37)

dec_tree.fit(x_train,y_train)


tree_pred = dec_tree.predict(x_test)
conf_mx = confusion_matrix(y_test, tree_pred)
print(f"    Positive    Negative    ")
print(f"       {conf_mx[0][0]}         {conf_mx[0][1]}")
print(f"       {conf_mx[1][0]}         {conf_mx[1][1]}")
print()
print(classification_report(y_test, tree_pred))

    Positive    Negative    
       109         42
       32         48

              precision    recall  f1-score   support

           0       0.77      0.72      0.75       151
           1       0.53      0.60      0.56        80

    accuracy                           0.68       231
   macro avg       0.65      0.66      0.66       231
weighted avg       0.69      0.68      0.68       231



In [36]:
from sklearn.model_selection import GridSearchCV, cross_val_score

"""
все еще существует риск переобучения на тестовом наборе, поскольку параметры можно настраивать до тех пор, пока оценщик не будет работать оптимально. Чтобы решить эту проблему, еще одна часть набора данных может быть представлена как так называемый «набор для проверки»: обучение продолжается на обучающем наборе, после чего выполняется оценка на проверочном наборе, и когда эксперимент кажется успешным. , окончательную оценку можно провести на тестовом наборе.

Однако, разбивая доступные данные на три набора, мы резко сокращаем количество выборок, которые можно использовать для обучения модели, а результаты могут зависеть от конкретного случайного выбора для пары наборов (обучение, проверка).

Набор тестов по-прежнему должен храниться для окончательной оценки, но набор для проверки больше не нужен при выполнении резюме. В базовом подходе, называемом k- кратным CV, обучающая выборка разбивается на k меньших наборов (другие подходы описаны ниже, но обычно следуют тем же принципам). Для каждой из k «фолдов» выполняется следующая процедура :

    Модель обучается с использованием  складок в качестве обучающих данных;
Результирующая модель проверяется на оставшейся части данных (т. е. она используется в качестве тестового набора для вычисления показателя производительности, такого как точность).

"""


##
##n_jobsint, default=None
##Number of jobs to run in parallel. None means 1 unless in a joblib.parallel_backend context. -1 means using all processors. See Glossary for more details.
##  
##

grid = GridSearchCV(dec_tree, {'max_depth': range(1,11), 
                               'max_features' : range(4,9)}, cv=5, n_jobs=-1, verbose=True)
grid.fit(x_train,y_train)
print(f"Best params: {grid.best_params_}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best params: {'max_depth': 4, 'max_features': 6}


In [37]:
grid_pred = grid.predict(x_test)
print(confusion_matrix(y_test, grid_pred))
print(classification_report(y_test,grid_pred))

[[115  36]
 [ 32  48]]
              precision    recall  f1-score   support

           0       0.78      0.76      0.77       151
           1       0.57      0.60      0.59        80

    accuracy                           0.71       231
   macro avg       0.68      0.68      0.68       231
weighted avg       0.71      0.71      0.71       231

